# Introduction to Data Science — Final Project
*MATH 4100 / COMP 5360, University of Utah, http://datasciencecourse.net/*


## Basic Information

Please fill in the following details:

- **Project Name:**  Guiding Optimizations with Gershgorin Discs    
- **Team Names:**  Joel Wood || Sasha Kushnir || Tymur Bolsheshapov
- **Emails:**  joel.wood@ngc.com || u1497176@utah.edu || klimkilov133@gmail.com   
- **UIDs:**  u0499977 ||  u1497176 || u1619352

## Background and Motivation

#### Applications and Use-cases:  
The fields of Operations Research and Convex Optimization form a cornerstone of applied mathematics, in which optimal combinations of variables and constraints are solved in order to max/min an objective function.   This form of optimization is used throughout many industries to help solve np-hard problems, by determining the best decisions in how to allocate resources which are governed by many complex constraints, such as:
- **Logistics:** - Solving "Traveling Salesman", "Knapsack", and "Vehicle Routing" problems to minimize shipping times, minimize number of trips, and reducing fuel usage.  Linear Programming was developed by George Danzig during WW2 to opimize the logistics of the US military, enabling the US to efficiently supply their operations with an unprecedented efficiency--the Simplex LP algorithm is considered one of the greatest innovations in applied mathematics.  Danzig is actually the inspiration of the movie "Good Will Hunting", in which, after arriving to class late, he "accidentally" solved two of the most famous unsolved statistics problems, which Danzig had mistakenly assumed were unusually difficult homework problems.
- **Finance:** - Optimizing portfolios with the goal to maximize returns for a specified level of risk ("efficient frontier").  It helps guide, based on risks and probabilities, how to optimally invest assets and develop stable cash-flows.
- **Manufacturing:** - Determining the optimal way to allocate resources across different product lines and determining optimal build sequences and batch sizing to maximize profits meeting customer demands while mitigating excess Inventory ("JIT Manufacturing").
- **Scheduling:** - Determining optimal schedules which consider both customer and employee needs, ensuring the right people are assigned where they are needed, while making sure schedules are compliant with labor laws.  OR assisted scheduling of flights and their crews substantially reduced flight costs and govern stochastic pricing strategies, slashing the cost of airfare for consumers and saving airlines tens of millions of dollars each year. 
- **Power Grids:** - Power grids have to consider the balance of electricity supply and demand across whole continents, in which fluctuations due to weather and other events require a balance between different power generation strategies (back-up generators) and building robust infrastructure.  This involves a balance of risk and costs, with catastrophic results from not properly balancing these many uncertain factors.

#### Opportunities:  
These problems explode exponentially, often exceeding billions of different combinations.  With binary variables themselves contributing to a combinatorial explosion, forming MILP (Mixed-Integer Linear Programs) which have more branch-and-bound tree leaves than there are atoms in the universe.  The memory and time requirements to solve many such problems become "impossible" to achieve with a generalized algorithm.  There is no "best" algorithm, and thus an open problem in these fields is better understanding how an optimal algorithm can be applied generally--similar to an agentic assignment of strategies.  Some common (but not inclusive) issues which require tuning to solve OR problems include:
- **Pivoting:** - Using Simplex LP, decision pivots move through the outer hull of the multi-dimensional space, often utilizing a "warm-start" in which a known viable configuration is used within a sparse system.   However, there are also Interior Point methods which seek to reduce iterations by traversing not the edge, but the interior of this hull.  These interior methods increase speed while being prone to matrix degeneracy, getting stuck at local optima, or becoming more resource intensive.
- **Pre-Solves:** - Many problems are initialized with a heuristic which will find an intersection of constraints and variables to eliminate redundant combinations (reducing the matrix size) and further reducing the problem space's feasible region.  This is tied to the "warm-start" in which you choice of a starting point will greatly impact on the number of iterations required, and also influence the ideal pivot techniques to traverse your redefined feasible region.
- **Numerical Stability:** - due to repeatedly needing to solve massive matrices, if the matrix is "ill-conditioned" where minor changes cause large fluctuations, the cost of individual calculations may become more problematic than simply doing additional stable calculations.  In these situations, controlling your matrix sparsity and numerical precision/stability becomes more important than reducing computations.
- **Slack and Hard-Constraints:** - Dual Simplex uses slack variables to better balance the variables to constraints, allowing for matrices which can "fail" to meet soft constraints with high penalties.  However, these slack variable penalties must be set high enough to enforce them, without causing numerical instability.  Sometimes setting a massive value will allow for fast solves, while sometimes it will cause abrupt crashes.  Similarly, within MILP problems, the use of constraints to shrink the feasible space may allow you to prune your branch and bound tree more effectively, but it may also make it more difficult to even find "feasible", much less optimal.  This creates a problem in which balance and structure of a matrix may make one combination solve in minutes, while another may take years (i.e. functionally impossible).   This makes generalized solvers difficult to make for even a particular type of problem.

Due to a lack of a generalized best algorithm and parameter balance for these problems, finding computationally efficient ways to understand underlying matrix structure and clustering variable types into "topologically" distinct groups allows for more guided and generalized ways to perform pre-solving for an ideal starting point within the feasible region, and also to improve pivot selections based on principles of "spectral analysis" to allow parallel or possibly convolutional computations across larger regions of the convex hull, as well as an understanding of degeneracy and sparsity fill-in risks from doing so.   It is believed that a stronger understanding of how to efficiently predict both structure and variable clustering within matrices will empower a more generalized approach to Convex Optimization, in which a more specialized approach can be assigned based on a computationally cheap preassessment.

One of the methods which may be powerful in the context of Operations Research in which there are massive, sparse, and positive definite matrices, is the use of Gershgorin Discs.   The calculation of Gershgorin discs forms circles within the complex plane, the union of these circles represent the region in which those variable's eigenvalues exist.  This is functionally useless in dense matrices or ill-conditioned matrices, as everything becomes a single massive union.  But properly balanced and sparse matrices will likely form distinct clusters of eigenvalues.  This method fails to find specific eigenvalues and does not reveal their vectors.  However, as seen below, this more generalized information can be derived at a much lower cost than computing out all the eigen pairs.
- **Gershgorin Discs:** - O(m+n) where m represents non-zero entries, in the sparse matrices found in most OR problems, this is functionally Amortized O(n), a linear algorithm. 
- **Gershgorin Bipartite Laplacian:** - due to OR usually having imbalances in the number of rows and columns (variables and constraints), you will need to form a new Bipartite Laplacian to be able to find eigenvalues for the system.  The formation of the Bipartite Laplacian is functionally O(m) within sparse matrices.  Because a Laplacian is diagonally dominant the calculation of Gershgorin Discs is further simplified.  This leads to an overall complexity of both algorithms to be approximately O(nm), more time consuming than finding the discs, but this will often be around Amortized O(n) or O(logn) within these sparse matrices.
- **Scaled Gershgorin Discs:** - due to the density of the RHS in these problems, if Gershgorin discs are not forming distinct unions, it is possible to use a Linear Program to find a diagonal scaling matrix which will adjust their radii or align them to their RHS values.  These scaling problems are generally much simpler to solve than the typical linear program, and within sparse matrices, they have a complexity of O(km) where k is the number of iterations required by the sub LP.  This number is often significantly less than m, often leading to a roughly linear complexity in sparse matrices.

#### Motivations:  
As seen, all of these algorithms have complexities which are significantly less than computing eigenpairs directly, which is O(n^3)—absolutely massive in a matrix with millions of rows and columns.  There are ways to compute a subset of needed eigenpairs within large sparse matrices, but they are similarly not reliable in their complexity, based on the very factors which influence Linear Programs.  Which therefore, the Gershgorin Disc unions, can lead to more efficient and targetted Eigenpair extraction if needed for a later optimization strategy.  It is therefore believed that this low cost, which is often around O(n), can be used to guide a generalized structure of predictive “optimal” optimization strategies within the fields of Operations Research and Convex Optimization.  If a probabilistic strategy can be developed to use Gershgorin Discs to make Linear Programs more generalized to solve problems more consistently, it has the potential to drive agentic improvements across many high-impact industries.


## Project Objectives

Our group members have an academic, professional, and personal interest in the intersection of statistics, operations research, differential equations and dynamic systems.  Our objective is to better understand matrix properties, structures, eigenvalues, and how these can be used to predict each other—with an overlap to random matrix theory.  The end goal of this project is to find new avenues of research for this team, to determine feasibility of existing research ideas, and to form a cornerstone of numerical analysis to justify and support further optimization research into the solving of industry relevant problems.

Our primary question is therefore, "Can you use Gershgorin Discs, to predict structure and properties of large, sparse, positive-definite matrices?  And can you similarly use any of these structures and properties to predict eigenvalue distributions within structured matrices?"  The answer to this question may provide an opportunity to develop generalized algorithm selection techniques, which would drive innovation of algorithms which provide billions of dollars in efficiency savings throughout society.  The current global economy is built on the scalability which Linear Programming has supported, allowing for a level of efficiency within a global system which would not be possible through naive computations.  

By unifying spectral/topological analysis with Linear Programming, it is believed that recent developments in Machine Learning can be unified with Linear Programming to improve both, and to further drive improved efficiencies within a plethora of fields.  This field of optimization is in high demand commerically, but academically it is often under-represented.  There are many open and unsolved problems in this domain, which represents a wealth of potential research topics and improvements beyond what is seen in many other fields.   We hope that the results of this project will give us insights and tools which can help us collectively find new optimization techniques--helping develop both applied mathematics and algorithmic improvements.


## Data Description

The testing of Matrices within Operations Research relies on a combination of sparsity and structure which is not easy to generate randomly.  Therefore, a set of standard test matrices are used based on real problems, supplemented with special random matrix generators based on these problems, if specific types of structures need to be further evaluated.  It is therefore industry standard that in evaluating algorithms, that these sorts of sets are used over generators--when evaulating generalized efficiencies.  Generators are usually used to evaluate more specialized algortithmic approaches.

#### Data Formats:  
These are collectively tied to netlib and the Matrix Market, which have been storing and developing standard matrix generators and sets for decades.  These matrices can be stored in the form of .mtx files, which can be read into python using the fast_matrix_market library in python which functions as an I/O for loading these matrix files; alternatively scipy.io.mmread and scipy.io.mmwrite can be used as well.
https://pypi.org/project/fast-matrix-market/

There are a few key resources which can be used to gather these matrix files, and to verify that parameter calculations and dimensionality reduction is being performed correctly:  

#### Matrix Sets:  
https://sparse.tamu.edu/
This website has over 2900 sparse matrix sets which are widely used as matrix benchmarks across a wide range of industries and fields.  They have Matrix Market files available for download as well as precalculated properties of the matrices, such as dimentionality, sparsity, symmetry, spectral rank, positive-definitive, Singular Value Decomposition (SVD) details, as well as some dimensionality reduction depictions, structure plots, and city plots.

The following groups are of particular interest:

##### Optimization Algorithms: 
- ***LPnetlib*** - This sparse matrix collection is commonly used to test algorithmic efficiencies in Linear Programming, generally built on business operations. (138 matrices)
- ***Schenk:*** - This matrix collection is filled with KKT Systems and Saddle-Point problems which often cause optimization challenges due to their poor diagonalization.  These problems appear unstable and indefinite, and often benefit from interior point methods which are robust against degeneracy. (5 matrices)
- ***GHS_indef:*** - These matrices are numerically challenging for standard solvers due to being indefinite with poor scaling, and high condition numbers.  This may provide an opportunity to assess if scaling of gershgorin discs is viable, or alternatively to use Brauer Cassini Ovals to further refine these unions.  This may allow for more stable pivots which consider complex rather than just real discs. (60 matrices)
##### Applied Physics & Engineering:
- ***HB:*** - This matrix collection is filled with a wide range of matrices used in engineering and physics across many different applied fields, ranging from small to large scale systems.  Particularly the Power Network subsets tend to form clumping and partitioning which may be useful for understanding Gershgorin Discs, and how they may help evaluate Structural Properties.  (292 matrices)
- ***Janna:*** - These are large finite element matricies which are often Symmetric Positive Definite, where discs may allow bounding the condition number, allowing for a cheap bound of the spectrum to tune parameters of pre-solving. (18 matrices)
- ***Rajat and Freescale:*** - These are Circuit Simulation matrices which have strong diagonal elements (due to the use of ground in circuit design), but form highly unsymmetric distributions.  These problems benefit from pivot and pre-solve optimizations. (32 + 8 matrices)
##### Network Graphs:
- ***LAW, SNAP, Pajek:*** - These are Social Network and Web Graph matrices which have loops and hubs which may provide challenges for using Discs without reordering or scaling.  Many of them are massive binary graphs (15 + 68 + 76 matrices)
- ***FlowIPM22:*** - These are Symmetric Diagonally Dominant Graph Laplacians, which makes them best for using Spectral Clustering to solve network flow problems. (11 matrices)


This gives us a sample of around 700 matrices which range from promising to problematic structures, with a focus on the types of matrices which are applied within industry.  If for some reason, it is determined that a generator is needed, there are a range of different generators available at:
https://math.nist.gov/MatrixMarket
However, many of these are based in FORTRAN or have applets, so the logic contained within may need to be restructured for python or generated outside of python.  These are likely unnecessary with the current scale of more accessible benchmark matrices.

## Ethical Considerations

The

### Data Cleaning and Processing

The established matrices can have their properties extracted either from computations within them and/or through web-scrapping the websites for this already calculated information.  It is proposed that we will use web-scrapping to extract the provided properties from the website, and then calculate the gershgorin discs (and other needed properties) through their matrix files using scripts.  This will result in a data set which contains our desired properties unified from web-scraping and calculation fill in, forming the dataset which we will perform analysis on.   We will store the Matrix files in case we need to perform additional analysis, such as PCA.


## Exploratory Analysis

Then

## Analysis Methodology

Also

## Project Schedule

And

In [1]:
# Code